# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [ ]:
# Step 1: Create your prompts
import requests
import ollama
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

#create message
messages = [
    {"role": "user", "content": 
     """
     How to fix error 403 in scrapping a website
     
   """
    }
]

#print the result
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])




In [17]:
# Import necessary libraries
import os
import requests
import ollama
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers, verify=False)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        
URL = Website("https://doanhnghiep.biz/3502447502")

def get_links_user_prompt(website):
    user_prompt = f"Here is the header of the website including the Name of the company {website.title} - "
    user_prompt += "please find all the relevant information about this company. Including name, address, Tax ID, Phone Number, legal rep, business type."
    user_prompt += "No need for the list of relevant companies\n"
    user_prompt += f" {website.text}"
    return user_prompt

REQUEST1 = get_links_user_prompt(URL)
messages1 = [{"role": "user", "content": REQUEST1}]
payload1 = {"model": MODEL, "messages": messages1, "stream": False}

response = requests.post(OLLAMA_API, json=payload1, headers=HEADERS)
#print(response.json()['message']['content'])
def Making_brochure(brochure):
    user_prompt = f"Here is the company profile {brochure} - "
    user_prompt += "Please make a 2 page brochure for the company. \n"
    return user_prompt

REQUEST2 = Making_brochure(response.json()['message']['content'])
messages2 = [ {"role": "user", "content": REQUEST2} ]
payload2 = {"model": MODEL, "messages": messages2, "stream": False}
response2 = requests.post(OLLAMA_API, json=payload2, headers=HEADERS)
print(response2.json()['message']['content'])

    

C:\Users\Khoa-Mac-PC\anaconda3\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doanhnghiep.biz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Dưới đây là nội dung của 2 trang brochure cho công ty CÔNG TY TNHH VSSC STEEL CENTER:

**Trang 1**

[ Hình ảnh công ty ]

Công Ty TNHH VSSC Steel Center

Một trong những nhà sản xuất hàng đầu về gia công cơ khí và xử lý kim loại tại Việt Nam

**Giới thiệu**

Công ty CÔNG TY TNHH VSSC STEEL CENTER được thành lập vào ngày 22/02/2021 với mục tiêu là trở thành một trong những nhà cung cấp hàng đầu về các sản phẩm cơ khí và xử lý kim loại tại Việt Nam. Chúng tôi cam kết mang đến cho khách hàng chất lượng tốt nhất và dịch vụ khách quan.

**Ngành nghề**

* Gia công cơ khí
* Xử lý và tráng phủ kim loại
* Bán buôn kim loại và quặng kim loại

**Sản phẩm**

Chúng tôi cung cấp các sản phẩm sau:

* Cơ khí: cọc thép, thanh thép, dây thép, v.v.
* Xử lý kim loại: xử lý bề mặt, xử lý nhiệt, v.v.
* Kim loại và quặng kim loại: bán buôn và cung ứng

**Địa chỉ**

Đường N7 Khu Công Nghiệp Phú Mỹ Ii, Phường Tân Phước, Thị Xã Phú Mỹ, Bà Rịa - Vũng Tàu

**Số điện thoại**

0254 3503 001

**Trang 2**

[ Hình ảnh

In [ ]:
"""
     create a python program to adding website + TaxID   
I have a list of TaxID as below     
List TaxID = 5300829397, 5300773641

The website is https://doanhnghiep.biz/

Example result is added to url list
https://doanhnghiep.biz/3502447502
https://doanhnghiep.biz/030352220
     """


# Define the base URL and list of TaxIDs
base_url = "https://doanhnghiep.biz/"
tax_ids = ["3502447502", "030352220"]

# Create an empty list to store the updated URLs
updated_urls = []

# Iterate over each TaxID and update the URL
for tax_id in tax_ids:
    updated_url = base_url + str(tax_id)
    updated_urls.append(updated_url)

# Print the updated URLs
print("Updated URLs:")
for url in updated_urls:
    print(url)